In [ ]:
Model = "gpt-3.5-turbo-1106"
Name = 'analysis2CoT'
split = 'dev'

SKIP_FIRST = 0
Queries = 5
sourcefile = f'./{split}.csv'
save_raw = f'./{split}_{Name}.txt'
save_file = f'./{split}_{Name}.csv'

Max_Tokens = 2000
openai_api_key = "your_api_key"
if '3.5' in Model:
    prices = (0.0015/1000, 0.002/1000) # input, output tokens
    WAIT = 60
elif '4' in Model:
    prices = (0.03/1000, 0.06/1000) # 0.06/0.002= 60/2=30. 
    WAIT = 60


Messages_correct = lambda input: [
    {
      "role": "system",
      "content": "You are an expert lawyer in the domain of U.S. civil procedure. You are given an introduction to a legal case, a question, an answer and an analysis that justifies the correctness of that answer. These are taken from a legal textbook, however they are not formal and they refer to multiple choice letters (A, B, C, D). Read the given analysis carefully and explain why the answer is correct in a short, formal, clear way, without mentioning anything about letters A, B, C, D. Reply with the format: \"Explanation:\\n<your_text>\", in one sentence. "
    },
    {
      "role": "user",
      "content": "Introduction:\nSection 1391(d), which defines the residence of a corporation that has contacts in one district within a state but not others, is confusing. Suppose, for example, that Omni-Plex Corporation has its principal place of business in the Northern District of California, sufficient contact to support general in personam jurisdiction over the corporation, that is, jurisdiction for a claim that arises anywhere. Assume further that Omni- Plex has no contacts in any other federal district within California. Under §1391(d), Omni-Plex ‘‘resides’’ in the Northern District of California, because, if the Northern District were a state, its contacts there would be sufficient to support personal jurisdiction over it there. But it would not ‘‘reside’’ in the Eastern District of California. It has no contacts there, so that, if the Eastern District were a state, it would not be subject to personal jurisdiction there. This is confusing because a defendant that is ‘‘at home’’ in a state (Daimler A.G. at 137) is subject to personal jurisdiction anywhere in the state, not just in the part of the state where the contacts exist. But §1391(d) tells us that, for venue purposes, we should look at the contacts in each district within the state separately. The corporation will be deemed to ‘‘reside’’ only in the districts where its contacts would support personal jurisdiction if that district were a state. Here’s a question to illustrate the operation of this vexing provision.\nQuestion:\n3. Manufacturing venue. Arthur wishes to bring a diversity action in federal court against Cleveland Manufacturing Company. Cleveland has its factory and principal place of business in the Northern District of Illinois, but no other contacts with Illinois. The claim is based on alleged negligence in making a toaster at the Illinois factory, which caused a fire in Arthur’s home in the Middle District of Georgia.\nAnswer:\nThe Southern District of Illinois is not a proper venue under §1391 because no events giving rise to the claim took place there and Cleveland does not reside there under the venue statute.\nAnalysis:\nSo, C is the right answer. The Southern District of Illinois is not a proper venue."
    },
    {
      "role": "assistant",
      "content": "Explanation:\nThe Southern District of Illinois cannot serve as a proper venue for Arthur's lawsuit against Cleveland Manufacturing Company under Section 1391(d) because Cleveland lacks sufficient contacts in this district to be considered as 'residing' there within the venue statute."
    },
    {
      "role": "user",
      "content": input
    },
  ]


Messages_incorrect = lambda input: [
    {
      "role": "system",
      "content": "You are an expert lawyer in the domain of U.S. civil procedure. You are given an introduction to a legal case, a question, a wrong answer and an analysis that justifies the reason the answer is incorrect. These are taken from a legal textbook, however they are not formal and they refer to multiple choice letters (A, B, C, D). Read the given analysis carefully and explain why the answer is incorrect in a short, formal, clear way, without mentioning anything about letters A, B, C, D. Reply with the format: \"Explanation:\\n<your_text>\", in one sentence. "
    },
    {
      "role": "user",
      "content": "Introduction:\nSection 1391(d), which defines the residence of a corporation that has contacts in one district within a state but not others, is confusing. Suppose, for example, that Omni-Plex Corporation has its principal place of business in the Northern District of California, sufficient contact to support general in personam jurisdiction over the corporation, that is, jurisdiction for a claim that arises anywhere. Assume further that Omni- Plex has no contacts in any other federal district within California. Under §1391(d), Omni-Plex ‘‘resides’’ in the Northern District of California, because, if the Northern District were a state, its contacts there would be sufficient to support personal jurisdiction over it there. But it would not ‘‘reside’’ in the Eastern District of California. It has no contacts there, so that, if the Eastern District were a state, it would not be subject to personal jurisdiction there. This is confusing because a defendant that is ‘‘at home’’ in a state (Daimler A.G. at 137) is subject to personal jurisdiction anywhere in the state, not just in the part of the state where the contacts exist. But §1391(d) tells us that, for venue purposes, we should look at the contacts in each district within the state separately. The corporation will be deemed to ‘‘reside’’ only in the districts where its contacts would support personal jurisdiction if that district were a state. Here’s a question to illustrate the operation of this vexing provision.\nQuestion:\n3. Manufacturing venue. Arthur wishes to bring a diversity action in federal court against Cleveland Manufacturing Company. Cleveland has its factory and principal place of business in the Northern District of Illinois, but no other contacts with Illinois. The claim is based on alleged negligence in making a toaster at the Illinois factory, which caused a fire in Arthur’s home in the Middle District of Georgia.\nAnswer:\nThe Southern District of Illinois is a proper venue under §1391(b)(2) because a substantial part of the events giving rise to the claim took place in Illinois.\nAnalysis:\nA is also dead on arrival. It relies on §1391(b)(2), which provides that venue is proper in a district where a substantial part of the events giving rise to the claim took place. But again, under §1391(b)(2) we have to focus on the district where the events took place, not the state. Arthur cannot obtain venue in the Southern District based on the fact that events giving rise to the claim took place in the Northern District."
    },
    {
      "role": "assistant",
      "content": "Explanation:\nVenue cannot be established in the Southern District of Illinois under §1391(b)(2) based on events that took place in another district, as venue should be focused on the district where the events giving rise to the claim took place, not the state."
    },    
    {
      "role": "user",
      "content": input
    },
  ]


In [ ]:
import os
import openai

openai.api_key = openai_api_key

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(3))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

def query(M):
  '''
  Openai api call
  '''
  response = completion_with_backoff(
      model=Model,
      messages= M,
      temperature=1,
      max_tokens=Max_Tokens,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
  )
  query = response.choices[0].message.content
  cost = prices[0]*response.usage.prompt_tokens + prices[1]*response.usage.completion_tokens
  return query, cost, response # extract_answer(query)

In [ ]:
import pandas as pd
import time

df = pd.read_csv(sourcefile)
cot = []
total=0
total_time=0
# Nones=[]
for index, row in df.iterrows():
    # idx,question,answer,label,analysis,complete analysis,explanation,cot
    intro = row['explanation']
    question = row['question']
    answer = row['answer']
    label = row['label']
    explanation = row['analysis']
    
    M = f'Introduction:\n{intro}Question:\n{question}\nAnswer:\n{answer}\nAnalysis:\n{explanation}'
    if label==1:
        messages = Messages_correct(M)
    elif label==0:
        messages = Messages_incorrect(M)
    else:
        print("label not binary")
        print(row['idx'])
        continue
    tries = 0
    while(tries>=0):
        try:
            ans, cost, response = query(messages)
            break
        except Exception as e:
            wait_min = 5
            print('Met RateLimit exception')
            print(f'Waiting {wait_min} minutes to avoid timeout...')
            time.sleep(wait_min*60)
            total_time+=wait_min*60
            print(f'Total time: {total_time/60:.0f} minutes')
            print('Resuming experiment:\n')
            tries -=1
    total+=1

    # save answer
    with open(save_raw, 'a') as sf:
        sf.write(str(row['idx']) + '\n' + ans + '\n')
    
    if ans.lower().find('explanation:')<0:
        print('PROBLEM with explanation')
        print(row['idx'])
        collected_ans = ans
    else:
        collected_ans = ans[ans.lower().find('explanation:')+len('explanation:'):].strip('\n')

    cot.append(collected_ans)

    if total% Queries==0 and WAIT>0:
        if total ==Queries:
            print("Analysis Example:")
            print(row['analysis'])
            print('-----------------------------------------------------------------------------')
            print(ans)
            print('######')
            print(collected_ans)            
        print(f'Waiting {WAIT} seconds to avoid timeout...')
        print(f'Total queries: {total}')
        time.sleep(WAIT)
        total_time+=WAIT
        print(f'Total time: {total_time/60:.0f} minutes')
        print('Resuming experiment\n')

df['cot'] = cot
df.to_csv(save_file, index=False)